Installation

In [ ]:
!pip install pdfplumber
!pip install camelot-py[cv]
!pip install --upgrade pymupdf
!pip install google-generativeai
!pip install faiss-cpu
!pip install transformers tqdm pandas pytesseract pillow easyocr langchain langchain-community langchain_openai faiss-cpu rank_bm25 pdf2image
!pip install sentence_transformers

Imports and set environment

In [ ]:
from dotenv import load_dotenv
import re
import os, glob
import pdfplumber
import camelot
import pymupdf
import numpy as np
import pandas as pd
from pathlib import Path
import google.generativeai as genai
import time
import faiss, json
import collections
import fitz
import io
from PIL import Image
import pytesseract
from langchain_core.documents import Document

load_dotenv()

# Agent CFO — Performance Optimization & Design

---
This is the starter notebook for your project. Follow the required structure below.  

You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.

## 1. Config & Secrets

In [ ]:
# Example:
# os.environ['GEMINI_API_KEY'] = 'your-key-here'
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

COMPANY_NAME = "Google"

def generate_test_log_path_name(base_path: str): 
    # create the directory if not exist 
    os.makedirs(base_path, exist_ok=True) 
    existing_files = [f for f in os.listdir(base_path) if f.startswith("test_") and f.endswith(".json")] 
    existing_indices = [int(f.split("_")[1].split(".")[0]) for f in existing_files if f.split("_")[1].split(".")[0].isdigit()] 
    next_index = max(existing_indices) + 1 if existing_indices else 1 

    return f"{base_path}/test_{next_index}.json"

## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


In [ ]:
DATA_DIR = "00-data"

# Annual reports (10-Ks)
annual_files = glob.glob(f"{DATA_DIR}/annuals/*.pdf")

# # Quarterly reports (10-Qs)
quarterly_files = glob.glob(f"{DATA_DIR}/quarterlies/*.pdf")

# # Press releases
# press_files = glob.glob(f"{DATA_DIR}/press_releases/*.pdf")

# Presentations
presentation_files = glob.glob(f"{DATA_DIR}/presentations/*.pdf")

# Supplements
supplement_files = glob.glob(f"{DATA_DIR}/supplements/*.pdf")

# # Transcripts
# transcript_files = glob.glob(f"{DATA_DIR}/transcripts/*.pdf")

In [ ]:
# for folder in ["annuals", "quarterlies", "press_releases", "presentations", "supplements", "transcripts"]:
for folder in ["annuals", "quarterlies", "presentations", "supplements"]:

    files = glob.glob(f"{DATA_DIR}/{folder}/*.pdf")
    print(f"{folder}: {len(files)} files")

## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

### Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, util

# load E5-base-v2
model = SentenceTransformer("intfloat/e5-base-v2")

def embed_text_query(s):
    # E5 expects prefix, and stripping/normalizing helps
    return model.encode(f"query: {s.strip().lower()}", normalize_embeddings=True)

def embed_text_passage(s) -> np.ndarray :
    """
    It returns a numpy ndarray of shape (len(s), embedding_dimension)
    [
        [0.012, -0.034, 0.089, ..., -0.045],   # embedding for passage 1
        [0.077,  0.120, -0.002, ...,  0.031],  # embedding for passage 2
        [0.050, -0.025,  0.061, ..., -0.018]   # embedding for passage 3
  ]
    """
    # E5 expects prefix, and stripping/normalizing helps
    return model.encode([f"passage: {chunk_text.strip().lower()}" for chunk_text in s],
                        convert_to_numpy=True,
                        normalize_embeddings=True,
                        show_progress_bar=True)

### Ingestion pipeline (Table)

In [ ]:
# Insert table ingestion code here

### Ingestion pipeline (Text)

In [ ]:
# --- Helpers ---
def clean_table(table):
    """Clean raw Camelot table output."""
    print ("Raw table:", table)
    return [
        [(cell or "").strip().replace("\n", " ") for cell in row]
        for row in table
    ]

def _normalize(s: str) -> str:
    s = (s or "").lower()
    # unify whitespace & quotes
    s = s.replace("\n", " ").replace("’", "'").replace("–", "-").replace("—", "-")
    s = " ".join(s.split())
    return s


def is_valid_table(table, numeric_threshold: float = 0.25) -> bool:
    """Return True if the table has enough numeric-looking cells to be considered real data."""
    if not table or not table[0]:
        return False

    cells = sum(len(r) for r in table)
    numeric_cells = 0
    num_pattern = re.compile(r"^\(?[+-]?\d[\d,\.]*\)?$")  # matches 5,439 or (1,200) etc.

    for row in table:
        for cell in row:
            cell = str(cell).strip().replace("$", "").replace("%", "")
            if num_pattern.match(cell):
                numeric_cells += 1

    return (numeric_cells / cells) >= numeric_threshold

In [ ]:
SECTION_EXAMPLES = {
    # --- Cover / Administrative ---
    "cover_page": [
        "united states securities and exchange commission form 10 k annual report pursuant to section 13 or 15d",
        "united states securities and exchange commission form 10 q quarterly report pursuant to section 13 or 15d",
        "cover page showing registrant name commission file number and state of incorporation",
        "front page identifying registrant address telephone number and fiscal year end",
    ],

    # --- Management Discussion ---
    "mdna": [
        "managements discussion and analysis of financial condition and results of operations",
        "md&a explaining liquidity capital resources and operating performance",
        "discussion and analysis of results of operations comparing current and prior periods",
        "analysis of changes in revenues costs cash flows and capital expenditures",
    ],

    # --- Risk Factors ---
    "risk_factors": [
        "risk factors that may affect future financial performance or share price",
        "discussion of material risks and uncertainties facing the company",
        "factors that could cause actual results to differ materially from forward looking statements",
    ],

    # --- Financial Highlights / Summary Data ---
    "summary_financial_data": [
        "selected financial data summarizing key performance indicators for the past five years",
        "summary of consolidated financial information and operating results",
        "selected financial highlights including revenue net income and earnings per share",
    ],

    # --- Income Statement ---
    "income_statement": [
        "consolidated statements of income showing revenue expenses and net income",
        "statement of operations or profit and loss reporting revenues and operating income",
        "consolidated statements of comprehensive income including other comprehensive income items",
        "income statement presenting total revenues cost of goods sold gross profit and net earnings",
    ],

    # --- Balance Sheet ---
    "balance_sheet": [
        "consolidated balance sheets showing assets liabilities and shareholders equity",
        "statement of financial position listing current assets long term liabilities and total equity",
        "balance sheet detailing cash accounts receivable inventories property plant and equipment",
    ],

    # --- Cash Flow Statement ---
    "cash_flow": [
        "consolidated statements of cash flows showing cash inflows and outflows from operating investing and financing activities",
        "statement of cash flows reconciling net income to net cash provided by operating activities",
        "cash flow statement detailing capital expenditures debt repayment and dividend payments",
    ],

    # --- Shareholders’ Equity ---
    "equity": [
        "consolidated statements of shareholders equity showing changes in retained earnings dividends and stock issuance",
        "statement of changes in stockholders equity presenting share repurchases and comprehensive income",
        "equity statement showing common stock treasury stock retained earnings and accumulated other comprehensive income",
    ],

    # --- Notes to Financial Statements ---
    "financial_statements": [
        "notes to consolidated financial statements providing accounting policies commitments contingencies and segment information",
        "footnotes accompanying consolidated financial statements describing significant accounting policies",
        "notes to financial statements detailing income taxes stock compensation and earnings per share",
        "supplementary information supporting consolidated financial statements",
    ],

    # --- Market Risk Disclosures ---
    "market_risk_disclosures": [
        "quantitative and qualitative disclosures about market risk",
        "discussion of exposure to interest rate foreign currency commodity and credit risk",
        "sensitivity analysis of market risk instruments",
    ],

    # --- Controls and Procedures ---
    "controls_procedures": [
        "controls and procedures section discussing disclosure controls and internal control over financial reporting",
        "evaluation of disclosure controls and procedures and changes in internal control",
        "managements report on internal control over financial reporting",
    ],

    # --- Legal Proceedings ---
    "legal_proceedings": [
        "description of material pending legal proceedings and litigation",
        "legal proceedings section detailing lawsuits claims and regulatory actions",
        "information about legal matters affecting the company",
    ],

    # --- Segment Information ---
    "segment_info": [
        "segment information describing operating segments geographic areas and major customers",
        "disclosure of business segments including revenue and profit by segment",
        "note providing details of segment performance and intersegment eliminations",
    ],

    # --- Signatures ---
    "signatures": [
        "signatures section signed on behalf of the registrant and principal officers",
        "signatures of directors executive officers and principal accounting officer",
        "signed by the registrant pursuant to the securities exchange act of 1934",
    ],

    # --- Exhibits ---
    "exhibits": [
        "exhibits and financial statement schedules",
        "list of exhibits and certifications required by form 10k or 10q",
        "exhibit index listing contracts and subsidiary information",
    ],

    # --- Fallback ---
    "other": [
        "miscellaneous sections not classified elsewhere including general disclosures appendices or cover letters",
    ],
}

In [ ]:
SECTION_EMBS = {
    sec: [embed_text_query(ex) for ex in examples]
    for sec, examples in SECTION_EXAMPLES.items()
}


def classify_section(text, table):
    page_text = _normalize(text)
    headers = _normalize(" ".join(table[0])) if table else ""
    first_col = _normalize(" ".join(row[0] for row in table[1:])) if table else ""

    combined = f"{page_text} {headers} {first_col}"
    emb = embed_text_query(combined)

    scores = {
        sec: max(util.cos_sim(emb, e).item() for e in embs)
        for sec, embs in SECTION_EMBS.items()
    }

    best = max(scores, key=scores.get)
    return best if scores[best] > 0.35 else "other"

In [ ]:
pdf_path = []
for folder in ["annuals", "quarterlies", "supplements"]:
    files = glob.glob(f"{DATA_DIR}/{folder}/*.pdf")
    pdf_path.extend(files)

print(f"Processing {len(pdf_path)} PDFs from all folders")
print("PDF paths:", pdf_path[:3], "...")

# keep track sections
sections = {}
output = {}

for pdfFile in pdf_path:
    pdf_name = os.path.basename(pdfFile)
    output[pdf_name] = {}

    print(f"\n=== Processing: {pdf_name} ===")

    # Step 1: extract raw text with pdfplumber
    with pdfplumber.open(pdfFile) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            section_text = classify_section(text, [[]])  # classifying page text only without tables
            sections[section_text] = sections.get(section_text, 0) + 1

            # Step 2: extract tables
            tables_pymupdf = extract_tables_from_page(pdfFile, i)

            tables = []
            for t in tables_pymupdf:
                df = t["dataframe"]
                raw_table = df.values.tolist()

                # if not is_valid_table(raw_table):
                #     # Skip tables that are mostly text, like footnotes or headers
                #     print(f"[SKIP] Page {i} – Non-numeric table filtered out")
                #     continue

                cleaned_table = clean_table(raw_table)
                section_table = classify_section(text, cleaned_table)

                # Track section counts
                sections[section_table] = sections.get(section_table, 0) + 1

                # Skip noise like signatures
                if section_table == "other" and "signature" in text.lower():
                    continue

                markdown_text = pd.DataFrame(cleaned_table).to_markdown(index=False)

                tables.append({
                    "section": section_table,
                    "header" : cleaned_table[0] if cleaned_table else [],
                    "rows" : cleaned_table[1:] if len(cleaned_table) > 1 else [],
                    "markdown": markdown_text
                })

            print(f"Page {i} → Text length: {len(text) if text else 0}, Tables Kept: {len(tables)}")

            output[pdf_name][i] = {
                "page_section": section_text,
                "text": text,
                "tables": tables
            }

print ("Section distribution:", sections)

# Step 3: Create directory if it doesn't exist and dump to JSON
output_path = f"{DATA_DIR}/test.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(output, f, indent=4)

print(f"\nOutput saved to: {output_path}")

In [ ]:
# Step 3: Create directory if it doesn't exist and dump to JSON
output_path = f"{DATA_DIR}/test.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(output, f, indent=4)

print(f"\nOutput saved to: {output_path}")

### Ingestion pipeline (Slides)

In [ ]:
def extract_slides_fitz(pdf_path, output_dir, lower_crop_extra=200):
    os.makedirs(output_dir, exist_ok=True)
    pdf = fitz.open(pdf_path)
    print(f"[INFO] Loaded '{pdf_path}' with {len(pdf)} pages.")

    for i, page in enumerate(pdf, start=1):
        pix = page.get_pixmap(dpi=200)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Crop lower for slides 1–2
        if i in [1, 2]:
            w, h = img.size
            crop_box = (0, 0, w, min(h + lower_crop_extra, h))
            img = img.crop(crop_box)

        img.save(os.path.join(output_dir, f"slide_{i:02d}.png"), "PNG")

    print(f"Extracted {len(pdf)} slides from {pdf_path}")

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
def ocr_folder(folder, label):
    docs = []
    for fname in sorted(os.listdir(folder)):
        if fname.endswith(".png"):
            path = os.path.join(folder, fname)
            text = pytesseract.image_to_string(Image.open(path))
            if text.strip():
                docs.append(Document(
                    page_content=text,
                    metadata={"image_path": path, "source_report": label}
                ))
    return docs

In [ ]:
# Extract slides and OCR
pdf_path = []
for folder in ["presentations"]:
    files = glob.glob(f"{DATA_DIR}/{folder}/*.pdf")
    pdf_path.extend(files)

print(f"Processing {len(pdf_path)} PDFs from all folders")

docs = []
for pdf in pdf_path:
    # Create output folder based on PDF name
    pdf_name = os.path.splitext(os.path.basename(pdf))[0]
    slide_folder = f"{DATA_DIR}/presentations/slides_{pdf_name}"
    
    print(f"Extracting slides from {pdf_name}...")
    extract_slides_fitz(pdf, slide_folder)
    
    # OCR the extracted slides
    pdf_docs = ocr_folder(slide_folder, pdf_name)
    docs.extend(pdf_docs)

print(f"Loaded {len(docs)} slide documents from {len(pdf_path)} PDFs.")

### Chunk (BASELINE)

#### Text + Tables

In [ ]:
# load the json file
with open(f"{DATA_DIR}/test.json", "r") as f:
    doc = json.load(f)

chunks = []

for fileDoc , docContent in doc.items():
    for page_num, content in docContent.items():
        page_section = content.get("page_section", "unknown")
        text = content.get("text", "")
        tables = content.get("tables", [])

        if text.strip():
            chunks.append({
                "id": f"{fileDoc}-page-{page_num}-text",
                "text": f"Financial filing text section: {text}",
                "metadata": {"document": fileDoc, "page_number": page_num, "page_section": page_section, "chunk_type": "prose"}
            })

        if tables:
            for t_index, table in enumerate(tables):
                    table_text = "\n".join([", ".join(row) for row in table.get("rows", [])])
                    table_markdown = table.get("markdown", "")

                    chunks.append({
                        "id": f"{fileDoc}-page-{page_num}-table-{t_index}",
                        "text": f"Financial statement table: {table_text}",
                        "markdown": table_markdown,
                        "metadata": {
                            "document": fileDoc,
                            "page_number": page_num,
                            "page_section": page_section,
                            "chunk_type": "table",
                            "table_index": t_index
                            }
                    })

print(f"Created {len(chunks)} chunks")

In [ ]:
text = [ chunk["text"] for chunk in chunks ]

embeddings = embed_text_passage (text)

print (f"Embeddings shape: {embeddings.shape}")

# Create a FAISS index - IP for normalized embeddings
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)
print (f"FAISS index contains {index.ntotal} vectors.")

# save it locally
output_dir = f"{DATA_DIR}/base"
os.makedirs(output_dir, exist_ok=True)

# storing the index
faiss.write_index(index, f"{output_dir}/base.faiss")
print(f"Index saved to {output_dir}/base.faiss")

# store the chunks
with open(f"{DATA_DIR}/base/chunks.json", "w") as f:
    json.dump(chunks, f, indent=4)

#### Slides

In [ ]:
# Extract slides and OCR (already extracted, just need to OCR)
docs_q1 = ocr_folder(f"slides_q1_2025", "Q1_2025")
docs_q2 = ocr_folder(f"slides_q2_2025", "Q2_2025")
docs = docs_q1 + docs_q2
print(f"Loaded {len(docs)} slide documents.")

slide_chunks = []
for doc in docs:
    # Extract slide number from filename (e.g., "slide_01.png" -> 1)
    filename = os.path.basename(doc.metadata['image_path'])
    slide_num = int(filename.split('_')[1].split('.')[0])
    
    slide_chunks.append({
        "id": f"{doc.metadata['source_report']}-slide-{slide_num:02d}",
        "text": doc.page_content,
        "metadata": {
            "source_report": doc.metadata['source_report'],
            "image_path": doc.metadata['image_path'],
            "slide_number": slide_num,
            "chunk_type": "slide"
        }
    })

print(f"Created {len(slide_chunks)} slide chunks.")

In [ ]:
slide_texts = [doc.page_content for doc in docs]

slide_embeddings = embed_text_passage (slide_texts)

print (f"Slide Embeddings shape: {slide_embeddings.shape}")

# Create a FAISS index - IP for normalized embeddings
slide_index = faiss.IndexFlatIP(slide_embeddings.shape[1])
slide_index.add(slide_embeddings)
print (f"FAISS index contains {slide_index.ntotal} vectors.")

# save it locally
output_dir = f"{DATA_DIR}/base"
os.makedirs(output_dir, exist_ok=True)

# storing the index
faiss.write_index(slide_index, f"{output_dir}/slides.faiss")
print(f"Index saved to {output_dir}/slides.faiss")

with open(f"{output_dir}/chunks_slides.json", "w") as f:
    json.dump(slide_chunks, f)

### Retrieval STAGE

In [ ]:
index = {}

def init_indexes():
    global index

    # Load base index
    documents_base_dir = f"{DATA_DIR}/base/base.faiss"
    index["index"] = faiss.read_index(documents_base_dir)
    index["chunks"] = json.load(open(f"{DATA_DIR}/base/chunks.json"))

    # Load slides index
    documents_slides_dir = f"{DATA_DIR}/base/slides.faiss"
    index["slides_index"] = faiss.read_index(documents_slides_dir)
    index["chunks_slides"] = json.load(open(f"{DATA_DIR}/base/chunks_slides.json"))

    print (f"chunks type : {type(index['chunks'])}, length: {len(index['chunks'])}")
    print (f"chunks_slides type : {type(index['chunks_slides'])}, length: {len(index['chunks_slides'])}")

def search_query(query, k=10):
    global index
    query_embedding = embed_text_query(query)  # Convert query to vector

    D, I = index["index"].search(np.array([query_embedding]), k=k)
    # D = distances/scores
    # I = indices of top k matching chunks

    results = [
        {
            "rank": rank + 1,
            "score": float(D[0][rank]),
            "text": index["chunks"][identified_chunk_idx]["text"],
            "markdown": index["chunks"][identified_chunk_idx].get("markdown", ""),
            "metadata": index["chunks"][identified_chunk_idx]["metadata"]
        }
        for rank, identified_chunk_idx in enumerate(I[0])
    ]




    return results

In [ ]:
def search_slides_query(query, k=3):
    global index
    query_embedding = embed_text_query(query)  # Convert query to vector

    D, I = index["slides_index"].search(np.array([query_embedding]), k=k)
    # D = distances/scores
    # I = indices of top k matching chunks

    results = [
        {
            "rank": rank + 1,
            "score": float(D[0][rank]),
            "text": index["chunks_slides"][identified_chunk_idx]["text"],
            "metadata": index["chunks_slides"][identified_chunk_idx]["metadata"]
        }
        for rank, identified_chunk_idx in enumerate(I[0])
    ]

    return results

#### Output print helper

In [ ]:
from textwrap import shorten
from tabulate import tabulate

def pretty_print_results(results, show_table=False):
    table_data = []
    for r in results:
        meta = r["metadata"]
        chunk_type = meta.get("chunk_type", "unknown")
        section = meta.get("page_section", "unknown")
        doc = meta.get("document", "unknown")
        page = meta.get("page_number", "?")
        score = f"{r['score']:.3f}"
        
        # shorten text for preview
        preview = shorten(r['text'], width=120, placeholder="…")
        table_data.append([r['rank'], score, chunk_type, section, doc, page, preview])

    headers = ["Rank", "Score", "Type", "Section", "Document", "Page", "Preview"]
    print(tabulate(table_data, headers=headers, tablefmt="github"))

    # print markdown tables
    if show_table:
        for r in results:
            if r["metadata"]["chunk_type"] == "table" and r.get("markdown"):
                print(f"\n Table from {r['metadata']['document']} (p.{r['metadata']['page_number']}):\n")
                print(r["markdown"])
                print("\n" + "-"*80 + "\n")


In [ ]:
# Initialize once
init_indexes()

# Search
results = search_query("Q1 2024 revenue", k=5)

# print(results)
pretty_print_results(results, show_table=True)

#### Slides Query

In [ ]:
from IPython.display import Image, display
import os

query = "2025 Q1 revenue"
results = search_slides_query(query, k=3)

for r in results:
    print(f"\n--- Result {r['rank']} ---")
    print(f"Score: {r['score']:.4f}")
    print("Caption:", r["text"])
    
    image_path = r["metadata"].get("image_path")
    if image_path and os.path.exists(image_path):
        display(Image(filename=image_path, width=600))
    else:
        print("(Image not found)")


## 4. Baseline Pipeline

### Agent Config and Imports

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.chains import LLMMathChain
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0) # consider 4o (will be a lot better but bye bye johnathan's credits)


# guiding prompt
system_prompt = """
You are a financial analyst assistant that can use tools.

When given retrieved data:
1. Identify all relevant components (e.g., for Operating Expenses: R&D, Sales & Marketing, G&A)
2. Extract the values for the requested years
3. Calculate using standard financial formulas
4. Cite each component source

If data is incomplete, state what's missing.
"""


### Context Formatter (logs the context also) (BASELINE VERSION)

In [ ]:
def format_context(query, results, use_markdown=False, base : bool = True, expanded_query: str = "") -> str :
    """
    Format retrieval results (from documents or slides) into a readable text context.
    Automatically detects the source type from metadata.
    """

    # save it locally 
    dir = f"{DATA_DIR}/logs/base" if base else f"{DATA_DIR}/logs/sections/" 
    file_name = generate_test_log_path_name(f"{dir}") 


    # add the query then save the results as json 
    with open(file_name, "w") as f: 
        json.dump({
            "query": query, 
            "expanded_query": expanded_query, 
            "results": results 
        }, f, indent=4)  
        
    parts = []
    for r in results:
        meta = r["metadata"]
        text = r.get("markdown") if use_markdown and r.get("markdown") else r["text"].strip()

        # Detect the type of source (document vs slide)
        if "document" in meta:
            doc = meta.get("document", "unknown")
            page = meta.get("page_number", "?")
            section = meta.get("page_section", "")
            header = f"[{doc}, page {page}] {section}".strip()
        elif "source_report" in meta:
            src = meta.get("source_report", "unknown report")
            slide = meta.get("slide_number", "?")
            header = f"[{src}, Slide {slide}]"
        else:
            header = "[unknown source]"

        parts.append(f"{header}\n{text}")

    return "\n\n---\n\n".join(parts)


## Table Search Function - To be replaced (Joc)

In [1]:
def search_tables_only(query: str, k: int = 5) -> str:
    """Retrieve ONLY tables for structured financial data"""
    search_k = k * 2
    
    query_embedding = embed_text_query(query)
    D, I = index["index"].search(np.array([query_embedding]), k=search_k)

    # Filter to keep ONLY tables
    table_results = []
    for rank, idx in enumerate(I[0]):
        if idx < len(index["chunks"]):
            chunk = index["chunks"][idx]
            if chunk['metadata'].get('chunk_type') == 'table':
                table_results.append({
                    'rank': rank + 1,
                    'score': float(D[0][rank]),
                    'text': chunk['text'],
                    'markdown': chunk.get('markdown', ''),
                    'metadata': chunk['metadata']
                })
                if len(table_results) >= k:
                    break
    
    return table_results 

def search_all_chunks(query: str) -> str:
    """Smart retrieval based on query type"""

    financial_terms = ['revenue', 'expense', 'income', 'margin', 'ratio', 'opex', 
                          'cost', 'profit', 'loss', 'cash flow']
    is_financial = any(term in query.lower() for term in financial_terms)

    #!!!!!!
    #!========================================================================================
                        #!REMOVING THE FINANCIAL PART FOR NOW TO TEST#!
    #!========================================================================================
    # if is_financial:
    #     print ("Detected financial query, retrieving tables only.") 
    #     # Dynamic k based on query complexity
    #     k = 10 if any(word in query.lower() for word in 
    #                  ["operating expense", "opex", "trend", "breakdown"]) else 5
    #     results = search_tables_only(query, k=k)
    
    # else: 
    #!======================================================================================== 
    #! ======================================================================================= 
    #!!!!!! 
    print ("Detected non-financial query, retrieving all chunks.")
    # Retrieve from both sources
    doc_results = search_query(query)
    slide_results = search_slides_query(query)

    results = doc_results + slide_results 
    print (f"Combining {len(doc_results)} doc results and {len(slide_results)} slide results.") 
    print ("="*60)
    print (f"combined Results : {results} ")
    print ("="*60)
    
    return format_context(query, results, use_markdown=True)         


#### Retriever Tool (BASELINE VERSION)

In [2]:
@tool("retriever", return_direct=False)
def retriever_tool(query: str) -> str:
    """  
    Use this tool to look up information in finance documents, spreadsheets, or reports.
    It retrieves the most relevant text or table snippets related to the given question.
    Especially helpful for questions like "total operating expenses" or "revenue growth by year".
    """
    results = search_query(query, k=5)
    return format_context(query, results, use_markdown=True)  

NameError: name 'tool' is not defined

#### Calculator Tool

In [ ]:
@tool("calculator", return_direct=False)
def calculator_tool(expression: str) -> str:
    """Safely evaluate a mathematical expression, e.g. (165 - 150) / 150 * 100."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error evaluating: {e}"


#### Agent -> ONE LLM call

In [ ]:
# Baseline Rule-based Agent
def rule_based_agent(query: str):
    """Simple rule-based routing - no LLM for tool selection"""
    
    # Rule 1: If query mentions calculation keywords -> retrieve + calculate
    calc_keywords = ['ratio', 'percentage', 'growth', 'change', 'calculate', 'compute']
    needs_calc = any(word in query.lower() for word in calc_keywords)
    
    retrieval_result = search_all_chunks(query)

    # Rule 3: If calculation needed, extract numbers and compute
    if needs_calc:
        # Simple extraction logic (you can enhance this)
        numbers = re.findall(r'\d+\.?\d*', retrieval_result)

    return retrieval_result

# Baseline pipeline
query = "Show Total Operating Expenses for 2022, 2023 and 2024."

query =  ["Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values.",
          "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison.",
          "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."]

query = query[0] 
# 1. Rule-based retrieval (no LLM)
retrieval_result = rule_based_agent(query)


# 2. ONE LLM call for final answer generation
final_response = llm.invoke([
    SystemMessage(content=system_prompt),
    HumanMessage(content=f"Query: {query}\n\nRetrieved Data:\n{retrieval_result}\n\nProvide final answer with citations in the required JSON format.")
])

# Results
print("=== Final Answer ===")
print(final_response.content)

#### Agent -> multiple LLM calls

In [ ]:
tools = [retriever_tool, calculator_tool]

#ReAct agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
# limit runtime + iterations
#!!! can be skipped since initialize_agent has alr returned a AgentExecutor 
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent.agent,
    tools=tools,
    max_iterations=3,
    max_execution_time=60
)

In [ ]:
query = "Show Total Operating Expenses for 2022, 2023 and 2024."

response = agent.invoke({
    "input": [
        SystemMessage(content=system_prompt),
        HumanMessage(content=query)
    ]
})
print(response["output"])


## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Gross Margin Trend (or NIM if Bank)
    *   Query: "Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values."
    *   Expected Output: A quarterly table of Gross Margin % (or NIM % if bank).

*   Operating Expenses (Opex) YoY for 3 Years
    *   Query: "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."
    *   Expected Output: A 3-year Opex table (absolute numbers and % change).

*   Operating Efficiency Ratio
    *   Query: "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."
    *   Expected Output: Table with Opex, Operating Income, and calculated ratio for 3 years.

In [ ]:
# TODO: Implement benchmark runner


## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [ ]:
# Example instrumentation schema
import pandas as pd
logs = pd.DataFrame(columns=['Query','T_ingest','T_retrieve','T_rerank','T_reason','T_generate','T_total','Tokens','CacheHits','Tools'])
logs

## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations


### EMBEDDINGS (USING OPENAI)

In [ ]:
from openai import OpenAI

client = OpenAI()

# choose your model
EMBED_MODEL = "text-embedding-3-small"  # or "text-embedding-3-large"

def embed_text_query(s):
    # E5 expects a prefix; same here for consistency
    text = f"query: {s.strip().lower()}"
    response = client.embeddings.create(model=EMBED_MODEL, input=text)
    return response.data[0].embedding

def embed_text_passage(chunks):
    # Normalize and add E5-like prefix to all passages
    inputs = [f"passage: {chunk.strip().lower()}" for chunk in chunks]
    response = client.embeddings.create(model=EMBED_MODEL, input=inputs)
    # convert it to numpy array
    embeddings = np.array([d.embedding for d in response.data])
    # return a list of numpy-like vectors
    return embeddings


### CHUNKING (SECTION INDEXES) (OPTIMIZED)

In [ ]:
from collections import defaultdict 
#*  Classify Chunks (Section/Hierachical Chunking)

 # turn each section into a faiss index
indexes = {} 
sections = defaultdict(list) 

# --- Group chunks by section ---
for c in chunks: 
    section = c["metadata"].get("page_section", "unknown") 
    sections[section].append(c) 

sections_path = f"{DATA_DIR}/sections" 


# build per section index 
count = 0 
for section , chunk_list in sections.items(): 
    print(f"Building index for section: {section} ({len(chunk_list)} chunks)")
    text = [ chunk["text"] for chunk in chunk_list ] 
    embeddings = embed_text_passage (text) 

    
    print (f"Embeddings shape for section {section}: {np.array(embeddings).shape}") 
    idx = faiss.IndexFlatIP(embeddings.shape[1])
    idx.add(embeddings)


    #os.makedirs(f"Google/sections/{section}", exist_ok=True)
    # 
    os.makedirs(f"{sections_path}/{section}", exist_ok=True )
      
    # storing the index 
    faiss.write_index(idx, f"{sections_path}/{section}/faiss_index_{section}.index")

    # storing the chunks 
    with open(f"{sections_path}/{section}/chunks_{section}.json", "w") as f:
        json.dump(chunk_list, f, indent=4)

    # storing the embeddings 
    np.save(f"{sections_path}/{section}/embeddings_{section}.npy", embeddings) 
    count += 1
    
print(f"✅ Built {count} FAISS sub-indexes.") 

## INIT INDEXES (SECTIONS )

In [ ]:
indexes = {} 


def init_indexes(): 
    """this dont need run if its in the same session"""

    
    documents_base_dir = f"{DATA_DIR}/sections" 
    sections_path = [os.path.join(documents_base_dir, f) for f in os.listdir(documents_base_dir) if os.path.isdir(os.path.join(documents_base_dir, f)) ]
    print ("Sections found:", sections_path) 

    for sec_path in sections_path: 
        section = os.path.basename(sec_path) 
        #print (f"Loading section: {section}") 

        # load faiss 
        idx = faiss.read_index(f"{sec_path}/faiss_index_{section}.index") 
        indexes[section] = {
            "index": idx,
            "chunks": json.load(open(f"{sec_path}/chunks_{section}.json")) 
        }

In [ ]:
init_indexes() 

print (f"✅ Loaded {len(indexes)} section FAISS indexes.") 

print (f" Chunk text preview : {indexes['income_statement']['chunks'][0]['text'][:100]}...")

### FORMATTING CONTEXT FOR SECTIONS INDEXES SEARCHES

In [ ]:
def format_context(query, results, use_markdown=False, base : bool = True, expanded_query: str = "") -> str :
    """
    Format retrieval results (from documents or slides) into a readable text context.
    Automatically detects the source type from metadata.
    """

    # save it locally 
    dir = f"{DATA_DIR}/logs/base" if base else f"{DATA_DIR}/logs/sections/" 
    file_name = generate_test_log_path_name(f"{dir}") 


    # add the query then save the results as json 
    with open(file_name, "w") as f: 
        json.dump({
            "query": query, 
            "expanded_query": expanded_query, 
            "results": results 
        }, f, indent=4)  
        
    parts = []
    for section_data in results:
        section = section_data.get("section", "unknown") 
        parts.append(f"## Section: {section}\n") 
        for r in section_data.get("ranking", []):
            meta = r["metadata"]
            text = r.get("markdown") if use_markdown and r.get("markdown") else r["text"].strip()

            # Detect the type of source (document vs slide)
            if "document" in meta:
                doc = meta.get("document", "unknown")
                page = meta.get("page_number", "?")
                section = meta.get("page_section", "")
                header = f"[{doc}, page {page}] {section}".strip()
            elif "source_report" in meta:
                src = meta.get("source_report", "unknown report")
                slide = meta.get("slide_number", "?")
                header = f"[{src}, Slide {slide}]"
            else:
                header = "[unknown source]"

            parts.append(f"{header}\n{text} ")

    return "\n\n---\n\n".join(parts)

### Search Query (Sections)

In [ ]:
def search_query(expanded_query, sections : list ,k=10):

    """
    Search across section-specific FAISS indexes for a given (already expanded) query.
    """

    print(f"[INFO] Searching {len(sections)} sections for: '{expanded_query}'")

    query_embedding = embed_text_query(expanded_query) 
    #D_all, I_all = [], [] 
    results = []


    for sec in sections: 
        if sec in indexes: 
            idx = indexes[sec]["index"]
            D, I = idx.search(np.array([query_embedding]), k=min(k, idx.ntotal))
            # D_all.append(D)
            # I_all.append(I)

            results.append(
                {
                    "section": sec, 
                    "ranking" : [
                        {
                            "rank": rank + 1 , 
                            "score": float(D[0][rank]), 
                            "text": indexes[sec]["chunks"][identified_chunk_idx]["text"], 
                            "metadata": indexes[sec]["chunks"][identified_chunk_idx]["metadata"] 
                        }   for rank, identified_chunk_idx in enumerate(I[0])           
                    ]
                }
            )
        else:
            print(f"[WARN] Section '{sec}' not found in indexes.") 

    print (f"[INFO] Completed search across sections.")
    print (f"results preview : {results[:1]} ... ")
    return results

### Internal Helper (EXPAND QUERY + choose sections)

In [ ]:
import re
import json

def choose_sections_for_query(query: str, available_sections: list) -> list:
    """Internal helper — not a tool."""
    sec_list = ", ".join(available_sections)
    prompt = f"""
    You are a financial data retrieval router.
    Given the user's question and the available 10-Q sections, 
    select the most relevant section(s) to search for an answer.

    Available sections: {sec_list}
    User question: {query}

    Return a JSON array of section names from the list above. Strictly start with '[' and end with ']'.
    Example output: ["income_statement", "balance_sheet"]
    """

    response = llm.invoke([SystemMessage(content=prompt)])

    try:
        selected = json.loads(response.content)
        if isinstance(selected, list):
            return [sec for sec in selected if sec in available_sections]
        return []
    except Exception:
        return []
    
def expand_query_for_retrieval(query: str) -> str:
    """Internal helper — not a tool."""
    prompt = f"""
    Expand the following financial query to improve retrieval from SEC filings.

    Guidelines:
    - Include synonyms (e.g., "operating expenses" → "SG&A", "total expenses").
    - Include both annual and quarterly phrasing (e.g., “fiscal year”, “quarter ended”).
    - Include context like "Consolidated Statements of Income", "Statements of Operations".
    - Keep it short but keyword-dense (2–3 sentences).

    User query: "{query}"
    """
    response = llm.invoke(prompt)
    return response.content.strip()


def safe_json_loads(s):
    """Tries to parse JSON even if it has weird hidden chars or minor formatting issues."""
    if not isinstance(s, str):
        return None
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        try:
            cleaned = (
                s.strip("` \n\t'\"")          # remove outer quotes / markdown ticks / newlines
                .replace("'", '"')            # replace single quotes with double
                .replace("\ufeff", "")        # remove BOM marker
            )
            # remove trailing commas or stray markdown
            cleaned = re.sub(r",\s*}", "}", cleaned)
            cleaned = re.sub(r",\s*]", "]", cleaned)
            return json.loads(cleaned)
        except Exception as e:
            print(f"[ERROR] safe_json_loads still failed: {e}")
            print(f"[DEBUG] Cleaned candidate: {repr(cleaned)}")
            return None


### RETRIEVER TOOL (internally choose sections and expand queries if llm agent didnt provide the sections)

In [ ]:

@tool("retriever", return_direct=False)
def retriever_tool(query : str ) -> str: 
    """
    Retrieve relevant text snippets from financial filings.

    Accepts:
    - A plain query string (e.g., "Show total operating expenses for 2023").
    - Or a JSON string with fields:
        query: financial question
        sections: list of section names such as ["income_statement", "balance_sheet"]

    If no sections are provided, the tool will automatically choose the most relevant ones.
    """

    available_sections = list(indexes.keys())

    print (f"[RETRIEVER_TOOL] agent provided query: {query}")
    print (f"[RETRIEVER_TOOL] query type: {type(query)}") 
    

    parsed = safe_json_loads(query)

    # 1️⃣ Try to parse JSON input if agent passes structured info
    if parsed:
        print(f"[INFO] Parsed agent input: {parsed}")
        query = parsed.get("query", query)
        sections = parsed.get("sections", None)
        print(f"[INFO] Agent provided query: {query}")
        print(f"[INFO] Agent provided sections: {sections}")
    else:
        print("[INFO] No structured sections provided, will auto-select.")
        sections = None

    # 1️⃣ Expand query internally
    expanded_query = expand_query_for_retrieval(query)

    # 2️⃣ If agent didn't pass sections, auto-select
    if not sections:
        sections = choose_sections_for_query(expanded_query, available_sections)
        print (f"[INFO] Auto selected sections: {sections}") 
        if not sections:
            sections = available_sections  # fallback to all sections 

    # 3️⃣ Perform retrieval
    results = search_query(expanded_query, sections, k=5) 
    return format_context(query, results, use_markdown=True, base=False, expanded_query=expanded_query) 


## Available Sections Retriever 

In [ ]:
@tool("available_sections_retriever", return_direct=False) 
def available_sections_retriever(query: str = "") -> list:
    """
    List all available sections in the indexed financial filings.

    Returns a JSON array of section names, e.g.:
    ["income_statement", "balance_sheet", "cash_flow", "mdna", "risk_factors"].

    Useful for understanding which sections can be queried via the retriever tool.
    """

    return list(indexes.keys()) 

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage

# 1️⃣ init indexes
init_indexes()

# 2️⃣ model
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 3️⃣ system message
system_prompt = """
You are a financial analyst assistant that can use tools.

When given retrieved data:
1. Identify all relevant components (e.g., for Operating Expenses: R&D, Sales & Marketing, G&A)
2. Extract the values for the requested years
3. Calculate using standard financial formulas
4. Cite each component source

If data is incomplete, state what's missing.
"""

# 4️⃣ tools
tools = [retriever_tool, calculator_tool, available_sections_retriever]

# 5️⃣ initialize agent (already returns an AgentExecutor)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs={"system_message": SystemMessage(content=system_prompt)}
)

agent.max_iterations = None 


In [ ]:
query = "Show Total Operating Expenses for 2022, 2023 and 2024."
query =  ["Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values.",
          "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison.",
          "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."]

query = query[2] 

response = agent.invoke({
    "input": query,
    '"query"': query   # notice the quotes *inside* the string key
})


print(response["output"])

## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib
